# Requirements¶

In [ ]:

# this variable will be used in `runner.train` and by default we disable FP16 mode

is_fp16_used = False

is_alchemy_used = False


# for augmentations

!pip install albumentations==0.4.3


# for pretrained segmentation models fo PyTorch

!pip install segmentation-models-pytorch==0.1.0


# for TTA

!pip install ttach==0.0.2

################

# Catalyst itself

!pip install -U catalyst

# For specific version of catalyst, uncomment:

# ! pip install git+http://github.com/catalyst-team/catalyst.git@{master/commit_hash}

################



# for tensorboard

!pip install tensorflow



# for alchemy experiment logging integration, uncomment this 2 lines below

# !pip install -U alchemy

# is_alchemy_used = True



# if Your machine support Apex FP16, uncomment this 3 lines below

# !git clone https://github.com/NVIDIA/apex

# !pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

# is_fp16_used = True




# Colab extras – Plotly



In [ ]:
import IPython


def configure_plotly_browser_state():

    display(IPython.core.display.HTML('''

        <script src="/static/components/requirejs/require.js"></script>

        <script>

          requirejs.config({

            paths: {

              base: '/static/base',

              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',

            },

          });

        </script>

        '''))


IPython.get_ipython().events.register('pre_run_cell', configure_plotly_browser_state)

Setting up GPUs

from typing import Callable, List, Tuple


import os

import torch

import catalyst


from catalyst.dl import utils


print(f"torch: {torch.__version__}, catalyst: {catalyst.__version__}")


# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # "" - CPU, "0" - 1 GPU, "0,1" - MultiGPU


SEED = 42

utils.set_global_seed(SEED)

utils.prepare_cudnn(deterministic=True)

# for alchemy experiment logging integration, uncomment this 2 lines below

!pip install -U alchemy

is_alchemy_used = True

if is_alchemy_used:

    monitoring_params = {

        "token": "abcde4532", # insert your personal token here

        "project": "segmentation_example",

        "group": "first_trials",

        "experiment": "first_experiment",

    }

    assert monitoring_params["token"] is not None

else:

    monitoring_params = None




# Dataset

Type Markdown and LaTeX: 𝛼2

usage: download-gdrive {FILE_ID} {FILENAME}

In [ ]:


%%bash


download-gdrive 1IIR4n_-8VlR8muHHRifNj0G1CyrL5FDd Final_Training.zip

extract-archive Final_Training.zip &>/dev/null

from pathlib import Path


ROOT = Path("Final_Training")


train_image_path = ROOT / "train3"

train_mask_path = ROOT / "train_masks3"

test_image_path = ROOT / "test"

Collect images and masks into variables.

ALL_IMAGES = sorted(train_image_path.glob("*.jpg"))

len(ALL_IMAGES)

ALL_MASKS = sorted(train_mask_path.glob("*.gif"))

len(ALL_MASKS)

print(ALL_MASKS)

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread as gif_imread
from catalyst import utils


def show_examples(name: str, image: np.ndarray, mask: np.ndarray):
    plt.figure(figsize=(10, 14))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title(f"Image: {name}")

    plt.subplot(1, 2, 2)
    plt.imshow(mask)
    plt.title(f"Mask: {name}")


def show(index: int, images: List[Path], masks: List[Path], transforms=None) -> None:
    image_path = images[index]
    name = image_path.name

    image = utils.imread(image_path)
    mask = gif_imread(masks[index])

    if transforms is not None:
        temp = transforms(image=image, mask=mask)
        image = temp["image"]
        mask = temp["mask"]

    show_examples(name, image, mask)

def show_random(images: List[Path], masks: List[Path], transforms=None) -> None:
    length = len(images)
    index = random.randint(0, length)
    show(index, images, masks, transforms)